In [29]:
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv('https://raw.githubusercontent.com/Jaydenzk/Video-Games-Sales/master/Video_Games_Sales_as_at_22_Dec_2016.csv')

In [31]:
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df = df.rename(columns={"Year_of_Release": "Year", "NA_Sales": "NA", "EU_Sales": "EU", "JP_Sales": "JP", "Other_Sales": "Other", "Global_Sales": "Global"})

In [33]:
df = df.dropna()

In [34]:
df.shape

(6825, 16)

In [35]:
df = df[df["Genre"].notnull()]

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [37]:
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [38]:
train.shape, validate.shape, test.shape

((4095, 16), (1365, 16), (1365, 16))

In [39]:
target = 'Global'
features = ['Platform', 'Genre']

X_train = train[features]
y_train = train[target]

X_validate = validate[features]
y_validate = validate[target]

X_test = test[features]
y_test = test[target]

X_train.shape, X_validate.shape, X_test.shape, y_train.shape, y_validate.shape, y_test.shape

((4095, 2), (1365, 2), (1365, 2), (4095,), (1365,), (1365,))

In [40]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline


pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestRegressor(n_estimators=100,
                          min_samples_split=8,                          
                          n_jobs=-1, random_state=42)

)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Platform', 'Genre'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'Platform',
                                          'data_type': dtype('O'),
                                          'mapping': PS2      1
GBA      2
PC       3
PS4      4
GC       5
3DS      6
X360     7
PS3      8
XB       9
Wii     10
PS      11
DS      12
WiiU    13
XOne    14
PSP     15
PSV     16
DC      17
NaN     -2
dtype: int64},
                                         {'col': 'Genre',
                                          'data_type': dtype('O'),
                                          'mappin...
                ('randomforestregressor',
                 RandomForestRegressor(bootstrap=True, criterion='mse',
                                       max_depth=None, max_feat

In [41]:
print('Validation Accuracy', pipeline.score(X_validate, y_validate))

Validation Accuracy 0.02996427318471317


In [42]:
from sklearn.metrics import classification_report

y_pred = pipeline.predict(X_test)

In [43]:
encoder = pipeline[0]
X_train = encoder.fit_transform(X_train)
X_val = encoder.transform(X_validate)
X_test = encoder.transform(X_test)
X_train.head()

,Platform,Genre
9938,1,1
5067,1,2
6861,2,3
14341,3,4
14002,4,1


In [44]:
from joblib import dump
dump(pipeline, 'pipeline.joblib')

['pipeline.joblib']